In [2]:
import redis
import re

import sys
from paths import Paths
paths = Paths()
sys.path.append(paths.scripts_root)
import util

%load_ext autoreload
%autoreload 2

# Display entire width of browser
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 95%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 99%; }
</style>
"""))

paths.print_paths()

Paths defined in paths module:
 paths.data_root: /Users/trafferty/data
 paths.notebook_root: /Users/trafferty/data/notebooks
 paths.scripts_root: /Users/trafferty/data/scripts
 paths.images_root: /Users/trafferty/data/images
 paths.projects_root: /Users/trafferty/data/projects
 paths.results_root: /Users/trafferty/data/results
 paths.tmp_root: /Users/trafferty/data/tmp
 paths.dat_root: /Users/trafferty/data/dat
 paths.tmp_dir: /Users/trafferty/tmp
 paths.ws_dir: /Users/trafferty/workspace
 paths.mii_home: /Users/trafferty/dev
 paths.inhouse: /Users/trafferty/dev/src/inhouse


In [3]:
try:
    r = redis.StrictRedis(host='localhost', port=6379, db=0, decode_responses=True)
    if r.ping():
        print("Connected to server")
except:
    print("Error: no connection to server")

Connected to server


In [5]:
# sandbox for db access

r.hgetall('2017-10-02_08.42.46:1:SNX1227354:14:FF031A:vars:1011')

{'Global Voltage': '18.200000',
 'Loop Count': '1011',
 'PD_delay_us': '210.000000',
 'Nozzle': '250.000000',
 'NozzleID': '250.000000',
 'Sample Clock': '105.000000',
 'Waveform': 'F690_DS.txt',
 'BitmapFile': 'recipe/g4x4shift_CANON_DIF.bmp',
 'Phase': 'C',
 'Row': '1.000000',
 'ImageFile': 'DispenserOpt_01011_2017-10-02_09.18.12.png'}

---
## Types of keys used in DIF

### Common Keys

##### experiments_started (set)
* Set containing the ExpRoots for all experiments that are in progress
* Removed once experiment has completed

##### experiments_completed (set)
* Set containing the ExpRoots for all experiments that have completed
* Will persist in set until results are removed

##### new_image_data (list)
* List of keyes of loop vars
* Keys pushed on after each experiment loop has completed, ie, image has been taken/saved, and ready for result processing

#### Keys produced at beginning of experiment

##### Experiement Root {expRoot}
* Root string for all keys in experiment
  * {expID}:{RunID}:{HeadSN}:{NozzleSize}:{MonimerID}

##### Base Variables (key pointing to hash)
* _key->_ {expRoot}:**base_vars**
* points to hash containing basic parameters of the experiment ("exp vars")
  * _ie:_ 2017-10-02_08.42.46:1:SNX1227354:14:FF031A:base_vars
* Example of hash:
```
                {'double_shutter': 'true',
                 'Experiment Type': 'DispenseOpt',
                 'Location': 'DIF Station',
                 'ImageFileRoot': '/media/DIF_synology/images/FF032A/14um/1227354/screening/2017-10-02_08.42.46',
                 'PrimaryNozzles': '3,0,6',
                 'HeadSerialNumber': 'SNX1227354',
                 'Magnification': '20x',
                 'Run Mode': 'Recipe',
                 'SecondaryNozzles': '1,2,4,5',
                 'DIF_Version': '0.2.0',
                 'double_shutter_delay_us': '4.000000',
                 'NozzleSize': '14',
                 'Camera': 'Andor',
                 'NumNozzles': '7',
                 'MonomerID': 'FF031A',
                 'git-sha': '96ec1353697c2452e06b3cb6fe30d97156ce9083',
                 'Total Loops': '4650',
                 'ExperimentID': '2017-10-02_08.42.46',
                 'git-branch': 'dif_branch'}
 ```

### Keys produced during experiment

##### Loop Variables (key pointing to hash)
* _key->_ {expRoot}:**vars**:{row}:{nozzle}:{loop_cnt}
* points to hash containing loop vars, ie, sample clock, voltage, PD delay, bitmap
  * _ie:_ 2017-10-02_08.42.46:1:SNX1227354:14:vars:FF031A:1:23:234
* Example of hash:
```
                {'Global Voltage': '18.200000',
                 'Loop Count': '1011',
                 'PD_delay_us': '210.000000',
                 'Nozzle': '250.000000',
                 'NozzleID': '250.000000',
                 'Sample Clock': '105.000000',
                 'Waveform': 'F690_DS.txt',
                 'BitmapFile': 'recipe/g4x4shift_CANON_DIF.bmp',
                 'Phase': 'C',
                 'Row': '1.000000',
                 'ImageFile': 'DispenserOpt_01011_2017-10-02_09.18.12.png'}
 ```
### Keys produced during result processing

##### Loop Results (hash)
* _key->_ {expRoot}:**res**:{row}:{nozzle}:{loop_cnt}
* points to hash containing loop vars, ie, velocity, volume ("loop vars")
  * _ie:_ 2017-10-02_08.42.46:1:SNX1227354:14:FF031A:res:1:23:234
* example of hash:
```
                {'gui_image_file': '/tmp/2_tmp_dif_file.png',
                 'Phase': 'C',
                 'Waveform': 'F661_DS.txt',
                 'Nozzle': '247',
                 'vel': '4.0643717678179678',
                 'volume': '0.55319984055761551',
                 'PD_delay_us': '220.000000',
                 'Row': '1',
                 'BitmapFile': 'recipe/grid3x4s1_DIF.bmp',
                 'Sample Clock': '115.000000',
                 'Global Voltage': '16.800000'}
```

#### *Results Set*
 Set containing all result keys (\*:res:\*) (see above)
* _key->_ {expRoot}:**results**

#### *Nozzles with results set*
 Set containing all nozzles that have results
* _key->_ {expRoot}:**row_noz_pairs**
* a row_noz_pair is a string representation of a float where the integer portion is the row and the decimal portion is the nozzle
  * _ie:_ 
    * 1.455 == nozzle 455 of row 1 (Xaar 100x)
    * 2.2 == nozzle 2 of row 2 (Xaar 100x or OCE)
    * 3.82 == nozzle 82 of row 3 (OCE)

#### Post-Processed Results

##### *Volume-Velocity Intersection Set*
 Sorted set containing intersection between range of volumes and range of 
* _key->_ {expRoot}:**vol_vel_inter**:{vol_low}-{vol_high}:{vel_low}-{vel_high}
* points to zset containing intersection between a range of 
  * _ie:_ 2017-10-02_08.42.46:1:SNX1227354:14:vars:FF031A:vol_vel_inter:0.293-0.343:6.000-7.500
    
##### *Volume-Velocity Intersection Set*
* _key->_ {expRoot}:**vol_vel_inter_by_vel**:{vol_low}-{vol_high}:{vel_low}-{vel_high}
* points to zset containing intersection between a range of 
  * _ie:_ 2017-10-02_08.42.46:1:SNX1227354:14:vars:FF031A:vol_vel_inter_by_vel:0.293-0.343:6.000-7.500
    
expRoot:vol_vel_inter_by_vel:0.288-0.348:6.000-7.500  type:  zset
expRoot:res_vol_set_desired  type:  zset
expRoot:vol_vel_inter_by_vel:0.293-0.343:5.000-6.500  type:  zset
expRoot:vol_vel_inter:0.293-0.343:1.373-2.276  type:  zset
expRoot:vol_vel_inter:0.288-0.348:1.438-2.211  type:  zset
expRoot:vol_vel_inter_by_vol:0.288-0.348:6.000-7.500  type:  zset
expRoot:res_vol_variance  type:  zset
expRoot:vol_vel_inter:0.288-0.348:6.000-7.500  type:  zset
expRoot:res_vel_set_desired  type:  zset
expRoot:vol_vel_inter:0.288-0.348:6.238-7.012  type:  zset
expRoot:vol_vel_inter_by_vol:0.293-0.343:5.000-6.500  type:  zset
expRoot:base_vars  type:  hash
expRoot:res_vel_variance  type:  zset
expRoot:res_vel_set  type:  zset
expRoot:res_vol_set  type:  zset

In [23]:
def extractExpRoot(key):
    expRoot_sections = 5
    return ':'.join(key.split(':')[0:(expRoot_sections)])

def extractExpID(key):
    expRoot = extractExpRoot(key)
    return expRoot.split(':')[0]

def extractRunID(key):
    expRoot = extractExpRoot(key)
    return expRoot.split(':')[1]

def extractHeadSN(key):
    expRoot = extractExpRoot(key)
    return expRoot.split(':')[2]

def extractNozzleID(key):
    expRoot = extractExpRoot(key)
    return expRoot.split(':')[3]

def extractMonomerID(key):
    expRoot = extractExpRoot(key)
    return expRoot.split(':')[4]

def findUniqueExpRoots(all_keys):
    expRoots = set()
    for k in all_keys:
        expRoots.add(extractExpRoot(k))
    return expRoots

In [16]:
# Here are some standard DIF keys
#
# set of all experiments that have started
experiments_started = r.smembers('experiments_started')

# set of all experiments that have completed
experiments_completed = r.smembers('experiments_completed')

# set of all result keys, ie, expRoot:res:row:nozzle:loop_cnt
results = r.smembers('results')

# set of all result keys, ie, expRoot:res:row:nozzle:loop_cnt
results = r.smembers('results')
results2 = r.keys(pattern='*:res:*')  # maybe should drop result set and just use this query?

print("Set of experiments started: ", experiments_started)
print("Set of experiments completed: ", experiments_completed)
print("Number of results: ", len(results))
print("Number of results: ", len(results2))


Set of experiments started:  {'2017-10-02_08.42.46:1:SNX1227354:14:FF031A'}
Set of experiments completed:  {'2017-10-02_08.42.46:1:SNX1227354:14:FF031A'}
Number of results:  11138
Number of results:  11138


In [28]:
all_keys = r.keys(pattern='*')
print("Number of keys: ", len(all_keys))
key = all_keys[0]
print("Key represents a", r.type(key))
print("Exp Root:  ", extractExpRoot(key))
print("Exp ID:    ", extractExpID(key))
print("Run ID:    ", extractRunID(key))
print("Head SN:   ", extractHeadSN(key))
print("Nozzle ID: ", extractNozzleID(key))
print("Monomer ID:", extractMonomerID(key))


Number of keys:  15814
Key represents a hash
Exp Root:   2017-10-02_08.42.46:1:SNX1227354:14:FF031A
Exp ID:     2017-10-02_08.42.46
Run ID:     1
Head SN:    SNX1227354
Nozzle ID:  14
Monomer ID: FF031A


In [52]:
expRoot_keys = []
expRoots = set()
nonExp_keys = []
vars_cnt = 0
res_cnt = 0

for k in all_keys:
    # if it is a proper Exp root, it will have at least 4 colons
    if len(re.findall(":", k)) >= 4:
        expRoot_keys.append(k)
        expRoots.add(extractExpRoot(k))
        
        if k.find(':vars:') >= 0:
            vars_cnt += 1
        elif k.find(':res:') >= 0:
            res_cnt += 1
        else:
            print("Not a var or res key:", k, " type: ", r.type(k))
    else:
        nonExp_keys.append(k)

print("------------------------\nNum expRoot keys: ", len(expRoot_keys))
print("Num unique expRoots keys: ", len(expRoots))
print("Num vars keys: ", vars_cnt)
print("Num res keys: ", res_cnt)
print("Exp Roots: ", expRoots)

print("Other keys: ", abs(len(expRoot_keys)-vars_cnt-res_cnt))
print("Non-exp keys:\n", nonExp_keys)

Not a var or res key: 2017-10-02_08.42.46:1:SNX1227354:14:FF031A:vol_vel_inter:0.293-0.343:6.000-7.500  type:  zset
Not a var or res key: 2017-10-02_08.42.46:1:SNX1227354:14:FF031A:vol_vel_inter_by_vel:0.288-0.348:6.000-7.500  type:  zset
Not a var or res key: 2017-10-02_08.42.46:1:SNX1227354:14:FF031A:res_vol_set_desired  type:  zset
Not a var or res key: 2017-10-02_08.42.46:1:SNX1227354:14:FF031A:vol_vel_inter_by_vel:0.293-0.343:5.000-6.500  type:  zset
Not a var or res key: 2017-10-02_08.42.46:1:SNX1227354:14:FF031A:vol_vel_inter:0.293-0.343:1.373-2.276  type:  zset
Not a var or res key: 2018-01-23_12.29.28:1:SNX1226841:18:FF031A  type:  hash
Not a var or res key: 2017-10-02_08.42.46:1:SNX1227354:14:FF031A:vol_vel_inter:0.288-0.348:1.438-2.211  type:  zset
Not a var or res key: 2017-10-02_08.42.46:1:SNX1227354:14:FF031A:vol_vel_inter_by_vol:0.288-0.348:6.000-7.500  type:  zset
Not a var or res key: 2017-10-02_08.42.46:1:SNX1227354:14:FF031A:res_vol_variance  type:  zset
Not a var or

In [50]:
k = all_keys[0]
print(k)
len(re.findall(":", k))
#re.findall(":", k)

2017-10-02_08.42.46:1:SNX1227354:14:FF031A:vars:1604


6

In [26]:
findUniqueExpRoots
len(expRoots)
expRoots

{'2017-10-02_08.42.46:1:SNX1227354:14', '2018-01-23_12.29.28:1:SNX1226841:18'}

In [21]:
k = all_keys[0]
':'.join(k.split(':')[0:4])

'2017-10-02_08.42.46:1:SNX1227354:14'

In [31]:
extractExpRoot('2017-10-02_08.42.46')

'2017-10-02_08.42.46'

In [53]:
len(r.keys(pattern="*:FF031A:*"))

15804

In [33]:
r.hgetall(r.keys(pattern="*:res:*")[0])

{'gui_image_file': '/tmp/3_tmp_dif_file.png',
 'Phase': 'C',
 'Waveform': 'F706_DS.txt',
 'Nozzle': '249',
 'vel': '7.0633515154795203',
 'volume': '0.24897811401724601',
 'PD_delay_us': '230.000000',
 'Row': '1',
 'BitmapFile': 'recipe/g4x4shift_CANON_DIF.bmp',
 'Sample Clock': '110.000000',
 'Global Voltage': '20.000000'}

In [ ]:
'2017-10-02_08.42.46:1:SNX1227354:14:vars:FF031A:vol_vel_inter:0.293-0.343:6.000-7.500'